# Processar Attributes

In [ ]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
df_input_path = '../../dados/X_testToronto.csv'

In [7]:
import numpy as np
import pandas as pd
import ast

In [9]:
def literal_eval(x):
    if pd.isna(x):
        return dict()
    raw_dict = ast.literal_eval(x)
    for key in raw_dict:
        raw_dict[key] = ast.literal_eval(raw_dict[key])
    return raw_dict
        
def proccess_df(df):
    #df['loc'] = df['loc'].apply(lambda x: ast.literal_eval(x))
    df['attributes'] = df['attributes'].apply(literal_eval)
    #df['hours'] = df['hours'].apply(lambda x: ast.literal_eval(x) if not pd.isna(x) else dict())

In [10]:
df = pd.read_csv(df_input_path)
proccess_df(df)
df_reg = df.copy()
df_reg = df_reg[['business_id', 'attributes']]
df_reg

,business_id,latitude,longitude,attributes,categories,hours
0,vHzWmPWHN4J1hRR3W3AMQg,43.648977,-79.420495,"{'Ambience': {'romantic': False, 'intimate': F...","Wine Bars, Tapas/Small Plates, Restaurants, Ba...","{'Monday': '18:0-23:0', 'Tuesday': '18:0-23:0'..."
1,15to24Q-otAHmto7FzsWRg,43.654002,-79.452189,"{'BusinessParking': {'garage': False, 'street'...","Beauty & Spas, Hair Salons, Barbers, Shopping,...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
2,8aqKdf4G4AAir8k_Kdslvg,43.648493,-79.383214,"{'ByAppointmentOnly': True, 'AcceptsInsurance'...","Health & Medical, Medical Centers","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
3,uxU1vr5AhhkTQ83X0bpeyg,43.772453,-79.448136,{'ByAppointmentOnly': False},"Health & Medical, Hospitals, Medical Centers",{}
4,f702hTJoqdR34Jn23C7d1A,43.665082,-79.460800,{'RestaurantsPriceRange2': 3},"Automotive, Bikes, Shopping, Auto Repair, Spor...",{}
...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,43.776237,-79.235318,"{'Caters': True, 'BikeParking': False, 'WiFi':...","Restaurants, Halal, Mexican","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
17578,o2811xZ0ufRi7WcsqWiGzw,43.670098,-79.388198,"{'RestaurantsPriceRange2': 2, 'BikeParking': F...","Food, Coffee & Tea",{}
17579,v9T9m_TbpLuaC2Kud5UbLA,43.679862,-79.340900,"{'RestaurantsPriceRange2': 2, 'BikeParking': T...","Sewing & Alterations, Shopping, Local Services","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
17580,bzGnAIe9OiBHGWJE-dZ_8g,43.745578,-79.220423,"{'BusinessParking': {'garage': False, 'street'...","Chicken Wings, Bars, Nightlife, Restaurants, P...","{'Monday': '11:30-1:0', 'Tuesday': '11:30-1:0'..."


### Transformar Attributes em dummy values

In [11]:
from collections import defaultdict

unique_attributes = defaultdict(set)
two_layer_attributtes = defaultdict(set)
attributes = df['attributes']
for item_dict in attributes:
    for key in item_dict:
        if type(item_dict[key]) is dict:
            for key2 in item_dict[key]:
                two_layer_attributtes[(key, key2)].add(item_dict[key][key2])
        else:
            unique_attributes[key].add(item_dict[key])

for attr1, attr2 in two_layer_attributtes:
    if attr1 in unique_attributes:
        unique_attributes.pop(attr1, None)

def dummy_attr(row):
    atributes = {}
    obj = row['attributes']
    if row['attributes'] == None:
        obj = dict()
    for key in unique_attributes:
        atributes[key] = None
        if key in obj:
            atributes[key] = obj[key]
            
    for attr1, attr2 in two_layer_attributtes:
        key = f'{attr1}.{attr2}'
        atributes[key] = None
        if attr1 in obj and type(obj[attr1]) is dict and attr2 in obj[attr1]:
            atributes[key] = obj[attr1][attr2]
    return atributes

df_attr = pd.DataFrame.from_dict(list(df_reg.apply(dummy_attr, axis=1)))
df_almost_dummie = pd.concat([df_reg, df_attr], axis=1)
new_columns = {at: 'attribute_' + at for at in df_attr.columns}
df_almost_dummie = df_almost_dummie.rename(columns=new_columns)

df_dummies = pd.get_dummies(df_almost_dummie[new_columns.values()].astype(str))
columns_to_drop = [x for x in list(df_dummies.columns) if ('_nan' in x or '_None' in x or '_none' in x)]
df_dummies = df_dummies.drop(columns_to_drop, axis=1)

df_reg = pd.concat([df_reg, df_dummies], axis=1)
df_reg

,business_id,latitude,longitude,attributes,categories,hours,attribute_Caters_False,attribute_Caters_True,attribute_WiFi_free,attribute_WiFi_no,...,attribute_DietaryRestrictions.vegan_True,attribute_DietaryRestrictions.kosher_False,attribute_DietaryRestrictions.halal_False,attribute_DietaryRestrictions.halal_True,attribute_DietaryRestrictions.soy-free_False,attribute_DietaryRestrictions.vegetarian_False,attribute_DietaryRestrictions.vegetarian_True,attribute_HairSpecializesIn.straightperms_False,attribute_HairSpecializesIn.africanamerican_False,attribute_HairSpecializesIn.asian_False
0,vHzWmPWHN4J1hRR3W3AMQg,43.648977,-79.420495,"{'Ambience': {'romantic': False, 'intimate': F...","Wine Bars, Tapas/Small Plates, Restaurants, Ba...","{'Monday': '18:0-23:0', 'Tuesday': '18:0-23:0'...",True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,15to24Q-otAHmto7FzsWRg,43.654002,-79.452189,"{'BusinessParking': {'garage': False, 'street'...","Beauty & Spas, Hair Salons, Barbers, Shopping,...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,8aqKdf4G4AAir8k_Kdslvg,43.648493,-79.383214,"{'ByAppointmentOnly': True, 'AcceptsInsurance'...","Health & Medical, Medical Centers","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,uxU1vr5AhhkTQ83X0bpeyg,43.772453,-79.448136,{'ByAppointmentOnly': False},"Health & Medical, Hospitals, Medical Centers",{},False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,f702hTJoqdR34Jn23C7d1A,43.665082,-79.460800,{'RestaurantsPriceRange2': 3},"Automotive, Bikes, Shopping, Auto Repair, Spor...",{},False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,43.776237,-79.235318,"{'Caters': True, 'BikeParking': False, 'WiFi':...","Restaurants, Halal, Mexican","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
17578,o2811xZ0ufRi7WcsqWiGzw,43.670098,-79.388198,"{'RestaurantsPriceRange2': 2, 'BikeParking': F...","Food, Coffee & Tea",{},False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17579,v9T9m_TbpLuaC2Kud5UbLA,43.679862,-79.340900,"{'RestaurantsPriceRange2': 2, 'BikeParking': T...","Sewing & Alterations, Shopping, Local Services","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'...",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17580,bzGnAIe9OiBHGWJE-dZ_8g,43.745578,-79.220423,"{'BusinessParking': {'garage': False, 'street'...","Chicken Wings, Bars, Nightlife, Restaurants, P...","{'Monday': '11:30-1:0', 'Tuesday': '11:30-1:0'...",False,True,False,True,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
attributes_columns = [x for x in df_reg.columns if x.startswith('attribute_')]
df_reg[attributes_columns] = df_reg[attributes_columns].apply(lambda x: x.astype(float))

In [ ]:
df_reg.to_parquet(product['data'])

## Teste manual

In [ ]:
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_reg.iloc[13]['attributes'])
    display(df_reg[attributes_columns].iloc[[13]])
    display('----------------------------------')
    display(df_reg.iloc[542]['attributes'])
    display(df_reg[attributes_columns].iloc[[542]])
    display('----------------------------------')
    display(df_reg.iloc[5389]['attributes'])
    display(df_reg[attributes_columns].iloc[[5389]])